In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

#!pip install yfinance
import yfinance as yf

#!pip install fredapi
from fredapi import Fred

#!pip install pandas-datareader pandas
import pandas_datareader.data as web

## Data scriping for Microsoft stock prices

In [2]:
msft = yf.Ticker("MSFT")

In [3]:
# Fast access to subset of stock info (opportunistic)
print(msft.fast_info)

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']


In [4]:
# Get historical market data
hist = msft.history(period="max")

# Show meta information about the history (requires history() to be called first)
msft.history_metadata

{'currency': 'USD',
 'symbol': 'MSFT',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': Timestamp('1986-03-13 09:30:00-0500', tz='America/New_York'),
 'regularMarketTime': Timestamp('2023-04-05 16:00:04-0400', tz='America/New_York'),
 'gmtoffset': -14400,
 'timezone': 'EDT',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 284.34,
 'chartPreviousClose': 0.097,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'start': Timestamp('2023-04-05 04:00:00-0400', tz='America/New_York'),
   'end': Timestamp('2023-04-05 09:30:00-0400', tz='America/New_York')},
  'regular': {'start': Timestamp('2023-04-05 09:30:00-0400', tz='America/New_York'),
   'end': Timestamp('2023-04-05 16:00:00-0400', tz='America/New_York')},
  'post': {'start': Timestamp('2023-04-05 16:00:00-0400', tz='America/New_York'),
   'end': Timestamp('2023-04-05 20:00:00-0400', tz='America/New_York')}},
 'dataGranularity': '1d',
 'range': '',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo

In [5]:
# Show actions (dividends, splits)
MSFT_actions = msft.actions

In [6]:
# Convert the index Date to a column.
MSFT_actions = MSFT_actions.reset_index(drop=False)
MSFT_actions

,Date,Dividends,Stock Splits
0,1987-09-21 00:00:00-04:00,0.00,2.0
1,1990-04-16 00:00:00-04:00,0.00,2.0
2,1991-06-27 00:00:00-04:00,0.00,1.5
3,1992-06-15 00:00:00-04:00,0.00,1.5
4,1994-05-23 00:00:00-04:00,0.00,2.0
...,...,...,...
81,2022-02-16 00:00:00-05:00,0.62,0.0
82,2022-05-18 00:00:00-04:00,0.62,0.0
83,2022-08-17 00:00:00-04:00,0.62,0.0
84,2022-11-16 00:00:00-05:00,0.68,0.0


In [7]:
# Keep only the year, month, and day of the Date column.
MSFT_actions["Date"] = pd.to_datetime(MSFT_actions["Date"].dt.strftime('%Y-%m-%d'))
MSFT_actions

,Date,Dividends,Stock Splits
0,1987-09-21,0.00,2.0
1,1990-04-16,0.00,2.0
2,1991-06-27,0.00,1.5
3,1992-06-15,0.00,1.5
4,1994-05-23,0.00,2.0
...,...,...,...
81,2022-02-16,0.62,0.0
82,2022-05-18,0.62,0.0
83,2022-08-17,0.62,0.0
84,2022-11-16,0.68,0.0


In [8]:
# Download the MSFT stock data.
MSFT_data = yf.download("MSFT", period = "max")

[*********************100%***********************]  1 of 1 completed


In [9]:
# Reset the index and move it to a column.
MSFT_data = MSFT_data.reset_index(drop=False)
MSFT_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.060657,1031788800
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.062823,308160000
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.063907,133171200
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.062281,67766400
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.061198,47894400
...,...,...,...,...,...,...,...
9337,2023-03-30,284.230011,284.459991,281.480011,284.049988,284.049988,25053400
9338,2023-03-31,283.730011,289.269989,283.000000,288.299988,288.299988,32740300
9339,2023-04-03,286.519989,288.269989,283.950012,287.230011,287.230011,24883300
9340,2023-04-04,287.230011,290.450012,285.670013,287.179993,287.179993,25802600


In [10]:
# Check if there are any missing values in MSFT_data.
MSFT_data.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [11]:
# Join the MSFT_data and MSFT_actions.
MSFT_data = MSFT_data.merge(MSFT_actions, on = "Date", how = "left")
MSFT_data

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.060657,1031788800,NaN,NaN
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.062823,308160000,NaN,NaN
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.063907,133171200,NaN,NaN
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.062281,67766400,NaN,NaN
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.061198,47894400,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9337,2023-03-30,284.230011,284.459991,281.480011,284.049988,284.049988,25053400,NaN,NaN
9338,2023-03-31,283.730011,289.269989,283.000000,288.299988,288.299988,32740300,NaN,NaN
9339,2023-04-03,286.519989,288.269989,283.950012,287.230011,287.230011,24883300,NaN,NaN
9340,2023-04-04,287.230011,290.450012,285.670013,287.179993,287.179993,25802600,NaN,NaN


In [13]:
# Fill NaNs with 0s.
MSFT_data = MSFT_data.fillna(0)
# Extract only the data between 1986-03-13 and 2023-03-31.
MSFT_data = MSFT_data[MSFT_data["Date"] <= "2023-03-31"]
MSFT_data

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.060657,1031788800,0.0,0.0
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.062823,308160000,0.0,0.0
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.063907,133171200,0.0,0.0
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.062281,67766400,0.0,0.0
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.061198,47894400,0.0,0.0
...,...,...,...,...,...,...,...,...,...
9334,2023-03-27,280.500000,281.459991,275.519989,276.380005,276.380005,26840200,0.0,0.0
9335,2023-03-28,275.790009,276.140015,272.049988,275.230011,275.230011,21878600,0.0,0.0
9336,2023-03-29,278.959991,281.140015,278.410004,280.510010,280.510010,25087000,0.0,0.0
9337,2023-03-30,284.230011,284.459991,281.480011,284.049988,284.049988,25053400,0.0,0.0


In [14]:
# Check if there are any missing values in MSFT_full.
MSFT_data.isna().sum()

Date            0
Open            0
High            0
Low             0
Close           0
Adj Close       0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [15]:
MSFT_data.to_csv("MSFT_data.csv", index=False)

## Data scriping for the US FED rate, GDP, and CPI

In [48]:
fred = Fred(api_key="ebf4aebe4861c90f2ef1f68e3667f511")

# Get the daily US Fed interest rate.
FED_rate_data = fred.get_series('DFF')

In [25]:
FED_rate_data = pd.DataFrame(FED_rate_data)
FED_rate_data = FED_rate_data.reset_index(drop=False)
FED_rate_data = FED_rate_data.rename(columns={"index": "Date", 0: "FED_rate"})
#FED_rate_data["Date"] = pd.to_datetime(FED_rate_data["Date"].dt.strftime('%Y-%m-%d'))
FED_rate_data = FED_rate_data[(FED_rate_data["Date"] >= "1986-03-13") & (FED_rate_data["Date"] <= "2023-03-31")]
FED_rate_data

,Date,FED_rate
11578,1986-03-13,7.56
11579,1986-03-14,7.39
11580,1986-03-15,7.39
11581,1986-03-16,7.39
11582,1986-03-17,7.60
...,...,...
25106,2023-03-27,4.83
25107,2023-03-28,4.83
25108,2023-03-29,4.83
25109,2023-03-30,4.83


In [28]:
# Extract only the data between 1986-03-13 and 2023-03-31.
# Set the start and end dates for the data.
start_date = "1986-01-01"
end_date = "2023-03-31"

# Use the FRED data source to get US GDP data
gdp_data = web.DataReader("GDP", "fred", start_date, end_date)

In [29]:
gdp_data = gdp_data.reset_index(drop=False)
gdp_data = gdp_data.rename(columns={"DATE": "Date"})
#gdp_data["Date"] = pd.to_datetime(gdp_data["Date"].dt.strftime('%Y-%m-%d'))
gdp_data

,Date,GDP
0,1986-01-01,4507.894
1,1986-04-01,4545.340
2,1986-07-01,4607.669
3,1986-10-01,4657.627
4,1987-01-01,4722.156
...,...,...
143,2021-10-01,24349.121
144,2022-01-01,24740.480
145,2022-04-01,25248.476
146,2022-07-01,25723.941


In [34]:
# Extract the consumer price index.
inflation_data = web.DataReader('CPIAUCSL', 'fred', start_date, end_date)

In [35]:
inflation_data = inflation_data.reset_index(drop=False)
inflation_data = inflation_data.rename(columns={"DATE": "Date", "CPIAUCSL": "CPI"})
#inflation_data["Date"] = pd.to_datetime(inflation_data["Date"].dt.strftime('%Y-%m-%d'))
inflation_data

,Date,CPI
0,1986-01-01,109.900
1,1986-02-01,109.700
2,1986-03-01,109.100
3,1986-04-01,108.700
4,1986-05-01,109.000
...,...,...
441,2022-10-01,297.987
442,2022-11-01,298.598
443,2022-12-01,298.990
444,2023-01-01,300.536


In [36]:
# Join the sp500_data and interest rate, GDP, CPI data.
MSFT_stats = MSFT_data.merge(FED_rate_data, on = "Date", how = "left")
MSFT_stats = MSFT_stats.merge(gdp_data, on = "Date", how = "left")
MSFT_stats = MSFT_stats.merge(inflation_data, on = "Date", how = "left")
MSFT_stats

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,FED_rate,GDP,CPI
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.060657,1031788800,0.0,0.0,7.56,NaN,NaN
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.062823,308160000,0.0,0.0,7.39,NaN,NaN
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.063907,133171200,0.0,0.0,7.60,NaN,NaN
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.062281,67766400,0.0,0.0,7.55,NaN,NaN
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.061198,47894400,0.0,0.0,7.38,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9334,2023-03-27,280.500000,281.459991,275.519989,276.380005,276.380005,26840200,0.0,0.0,4.83,NaN,NaN
9335,2023-03-28,275.790009,276.140015,272.049988,275.230011,275.230011,21878600,0.0,0.0,4.83,NaN,NaN
9336,2023-03-29,278.959991,281.140015,278.410004,280.510010,280.510010,25087000,0.0,0.0,4.83,NaN,NaN
9337,2023-03-30,284.230011,284.459991,281.480011,284.049988,284.049988,25053400,0.0,0.0,4.83,NaN,NaN


In [40]:
# Manually add in the row for 1986-03-01 in order to get the first value for GDP and CPI.
# This is for fill in the NAs in GDP and CPI: since they are recorded on a quarterly/monthly basis, 
# the days in between two records are filled in using the first record.
df2 = pd.DataFrame([[inflation_data["Date"][2], 
                     np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
                     gdp_data[gdp_data["Date"] == "1986-01-01"]["GDP"].item(), 
                     inflation_data[inflation_data["Date"] == "1986-03-01"]["CPI"].item()]], 
                   columns=MSFT_stats.columns)
df2
MSFT_stats = pd.concat([df2, MSFT_stats])
MSFT_stats

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,FED_rate,GDP,CPI
0,1986-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4507.894,109.1
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.060657,1.031789e+09,0.0,0.0,7.56,NaN,NaN
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.062823,3.081600e+08,0.0,0.0,7.39,NaN,NaN
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.063907,1.331712e+08,0.0,0.0,7.60,NaN,NaN
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.062281,6.776640e+07,0.0,0.0,7.55,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9334,2023-03-27,280.500000,281.459991,275.519989,276.380005,276.380005,2.684020e+07,0.0,0.0,4.83,NaN,NaN
9335,2023-03-28,275.790009,276.140015,272.049988,275.230011,275.230011,2.187860e+07,0.0,0.0,4.83,NaN,NaN
9336,2023-03-29,278.959991,281.140015,278.410004,280.510010,280.510010,2.508700e+07,0.0,0.0,4.83,NaN,NaN
9337,2023-03-30,284.230011,284.459991,281.480011,284.049988,284.049988,2.505340e+07,0.0,0.0,4.83,NaN,NaN


In [41]:
def fill_NAs(col):
    for i in range(len(col)):
        if np.isnan(col[i]):
            col[i] = col[i-1]

gdp = list(MSFT_stats["GDP"])
fill_NAs(gdp)
cpi = list(MSFT_stats["CPI"])
fill_NAs(cpi)

MSFT_stats["GDP"] = gdp
MSFT_stats["CPI"] = cpi

In [42]:
# Exclude the first row after filling in NAs.
MSFT_stats = MSFT_stats[1:]
MSFT_stats

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,FED_rate,GDP,CPI
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.060657,1.031789e+09,0.0,0.0,7.56,4507.894,109.100
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.062823,3.081600e+08,0.0,0.0,7.39,4507.894,109.100
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.063907,1.331712e+08,0.0,0.0,7.60,4507.894,109.100
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.062281,6.776640e+07,0.0,0.0,7.55,4507.894,109.100
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.061198,4.789440e+07,0.0,0.0,7.38,4507.894,109.100
...,...,...,...,...,...,...,...,...,...,...,...,...
9334,2023-03-27,280.500000,281.459991,275.519989,276.380005,276.380005,2.684020e+07,0.0,0.0,4.83,25723.941,301.648
9335,2023-03-28,275.790009,276.140015,272.049988,275.230011,275.230011,2.187860e+07,0.0,0.0,4.83,25723.941,301.648
9336,2023-03-29,278.959991,281.140015,278.410004,280.510010,280.510010,2.508700e+07,0.0,0.0,4.83,25723.941,301.648
9337,2023-03-30,284.230011,284.459991,281.480011,284.049988,284.049988,2.505340e+07,0.0,0.0,4.83,25723.941,301.648


In [43]:
# Check the NAs in the final dataframe.
MSFT_stats.isna().sum()

Date            0
Open            0
High            0
Low             0
Close           0
Adj Close       0
Volume          0
Dividends       0
Stock Splits    0
FED_rate        0
GDP             0
CPI             0
dtype: int64

In [46]:
MSFT_stats.to_csv("MSFT_stats.csv", index=False)